## Calculate land cover trends

Prepare temporal statistics (trend, intercept, stdev) data at the co2-data resolution

Land cover types: 
- individual types 
- total forest 
- shrub+savanna 
- grassland 
- cropland 
- grassland+cropland+CropNatMosiac  


In [16]:
# Import packages
import os
import rioxarray as rio
import xarray as xr
from xarrayutils.utils import linear_trend

### General Settings

In [17]:
output_dir = '../data_output/'
inversion = 's99'
lc_proc_path = '/Users/moyanofe/BigData/GeoSpatial/LandCover/LandCover_MODIS_MCD12/MCD12C1_proc'
file_in = 'MCD12C1.A2001-2021.061.LCtype1.All.lr.nc'

### Load data

In [18]:
# Load the co2 data ----
file_co2amp_trend = os.path.join(output_dir, 'co2invSeasAmpTrend_' + inversion + '.nc')
co2amp = rio.open_rasterio(file_co2amp_trend)
co2amp.rio.write_crs(4326, inplace=True)

<xarray.Dataset>
Dimensions:      (band: 1, x: 144, y: 87)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -178.8 -176.2 -173.8 -171.2 ... 173.8 176.2 178.8
  * y            (y) float64 83.0 81.0 79.0 77.0 ... -83.0 -85.0 -87.0 -89.0
    spatial_ref  int64 0
Data variables:
    slope        (band, y, x) float64 ...
    intercept    (band, y, x) float64 ...
    r_value      (band, y, x) float64 ...
    p_value      (band, y, x) float64 ...
    std_err      (band, y, x) float64 ...

### Preparing LC data

In [19]:

# Index for MCD12C1 Land_Cover_Type_1_Percent: IGBP land cover types
lcIndex = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
lcNames = ['Water', 'ENForest', 'EBForest', 'DNForest', 'DBForest', 
    'MixForest', 'ClosedShrub', 'OpenShrub', 'WoodySavanna',
    'Savanna', 'Grassland', 'PermWetland', 'Cropland',
    'Urban', 'CropNatMosiac', 'PermSnowIce', 'Barren']

### Calculating trends

The linear_trend function from xarrayutils.utils returns an xarray dataset with variables including slope, intercept, p-value


In [20]:
# Load the low resolution LC data

filepath_in = os.path.join(lc_proc_path, file_in)
ds_lc = rio.open_rasterio(filepath_in, masked=True)


# Calculate trends and intercept for each LC type

ds_lcTrends = xr.Dataset()

for i in range(len(lcNames)): # [0]: # range(len(lcNames)):
    # print(lcNames[i])
    da_lc = ds_lc[lcNames[i]]
    # Calculate trends
    lc_trend = linear_trend(da_lc, 'time')
    lc_trend = lc_trend.to_array()
    lc_trend.attrs['units'] = 'percent/y'  # Should change to an array with units for each variable
    lc_trend.attrs['long_name'] = 'Trend in land cover' # Should change to an array with names for each variable
    lc_trend.rio.write_crs(4326, inplace=True)
    ds_lcTrends[lcNames[i]] = lc_trend


# Calculate trends for grouped LC types

lcNamesNew = ['Forest', 'Shrub', 'GrassCrop']
ds_lc['Forest'] = ds_lc['ENForest'] + ds_lc['EBForest'] + ds_lc['DNForest'] + ds_lc['DBForest'] + + ds_lc['WoodySavanna']
ds_lc['Shrub'] = ds_lc['ClosedShrub'] + ds_lc['OpenShrub']
ds_lc['GrassCrop'] = ds_lc['Grassland'] + ds_lc['Cropland'] + ds_lc['CropNatMosiac']

ds_lcTrends = xr.Dataset()

for i in range(len(lcNamesNew)): # [0]: # range(len(lcNamesNew)):
    # print(lcNamesNew[i])
    da_lc = ds_lc[lcNamesNew[i]]
    # Calculate trends
    lc_trend = linear_trend(da_lc, 'time')
    lc_trend = lc_trend.to_array()
    lc_trend.attrs['units'] = 'percent/y'  # Should change to an array with units for each variable
    lc_trend.attrs['long_name'] = 'Trend in land cover' # Should change to an array with names for each variable
    lc_trend.rio.write_crs(4326, inplace=True)
    ds_lcTrends[lcNamesNew[i]] = lc_trend


# Save trends to file

file_out = os.path.join(output_dir, 'MCD12C1.A2001-2021.061.LCtype1.lr.Trends.nc')
ds_lcTrends.to_netcdf(file_out)